# Описание проекта

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

# 1. Загрузка и подготовка данных

- Загрузим необходимые библиотеки, датасет и получим общую информацию

In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
import numpy as np
import random
from sklearn.preprocessing import StandardScaler


In [2]:
#data_0 = pd.read_csv('/Users/1/Documents/2019_05_29/geo_data_0.csv')
#data_1 = pd.read_csv('/Users/1/Documents/2019_05_29/geo_data_1.csv')
#data_2 = pd.read_csv('/Users/1/Documents/2019_05_29/geo_data_2.csv')
data_0 = pd.read_csv('/datasets/geo_data_0.csv')
data_1 = pd.read_csv('/datasets/geo_data_1.csv')
data_2 = pd.read_csv('/datasets/geo_data_2.csv')

In [3]:
print(data_0.info())
data_0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None


,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647
...,...,...,...,...,...
99995,DLsed,0.971957,0.370953,6.075346,110.744026
99996,QKivN,1.392429,-0.382606,1.273912,122.346843
99997,3rnvd,1.029585,0.018787,-1.348308,64.375443
99998,7kl59,0.998163,-0.528582,1.583869,74.040764


In [4]:
print(data_1.info())
data_1

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305
...,...,...,...,...,...
99995,QywKC,9.535637,-6.878139,1.998296,53.906522
99996,ptvty,-10.160631,-12.558096,5.005581,137.945408
99997,09gWa,-7.378891,-3.084104,4.998651,137.945408
99998,rqwUm,0.665714,-6.152593,1.000146,30.132364


In [5]:
print(data_2.info())
data_2

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746
...,...,...,...,...,...
99995,4GxBu,-1.777037,1.125220,6.263374,172.327046
99996,YKFjq,-1.261523,-0.894828,2.524545,138.748846
99997,tKPY3,-1.199934,-2.957637,5.219411,157.080080
99998,nmxp2,-2.419896,2.417221,-5.548444,51.795253


### Изучив общую информацию по датасетам, мы имеем:
1. Имеется 3 датасета, состоищих из 100000 строк каждая;
2. Всего параметров в датасете 5. 
3. Все параметры, кроме id, имеют численные значения.
4. Пропусков нет

- Проверим датасет на наличие дубликатов:

In [6]:
data_0['id'].nunique()

99990

In [7]:
dublicate_0 = (
    data_0
    .pivot_table(index=('id'), values='f0', aggfunc='count')
    .reset_index()
    .query('f0 > 1')
)
dublicate_0 = dublicate_0['id'].tolist()
dublicate_0

['74z30',
 'A5aEY',
 'AGS9W',
 'HZww2',
 'QcMuo',
 'Tdehs',
 'TtcGQ',
 'bsk9y',
 'bxg6G',
 'fiKDv']

In [8]:
print(data_0.query('id in @dublicate_0').sort_values(by = 'id'))

          id        f0        f1         f2     product
66136  74z30  1.084962 -0.312358   6.990771  127.643327
64022  74z30  0.741456  0.459229   5.153109  140.771492
51970  A5aEY -0.180335  0.935548  -2.094773   33.020205
3389   A5aEY -0.039949  0.156872   0.209861   89.249364
69163  AGS9W -0.933795  0.116194  -3.655896   19.230453
42529  AGS9W  1.454747 -0.479651   0.683380  126.370504
931    HZww2  0.755284  0.368511   1.863211   30.681774
7530   HZww2  1.061194 -0.373969  10.430210  158.828695
63593  QcMuo  0.635635 -0.473422   0.862670   64.578675
1949   QcMuo  0.506563 -0.323775  -2.215583   75.496502
75715  Tdehs  0.112079  0.430296   3.218993   60.964018
21426  Tdehs  0.829407  0.298807  -0.049563   96.035308
92341  TtcGQ  0.110711  1.022689   0.911381  101.318008
60140  TtcGQ  0.569276 -0.104876   6.440215   85.350186
89582  bsk9y  0.398908 -0.400253  10.122376  163.433078
97785  bsk9y  0.378429  0.005837   0.160827  160.637302
41724  bxg6G -0.823752  0.546319   3.630479   93

In [9]:
data_1['id'].nunique()


99996

In [10]:
dublicate_1 = (
    data_1
    .pivot_table(index=('id'), values='f0', aggfunc='count')
    .reset_index()
    .query('f0 > 1')
)
dublicate_1 = dublicate_1['id'].tolist()
dublicate_1

['5ltQ6', 'LHZR0', 'bfPNe', 'wt4Uk']

In [11]:
print(data_1.query('id in @dublicate_1').sort_values(by = 'id'))

          id         f0         f1        f2     product
5849   5ltQ6  -3.435401 -12.296043  1.999796   57.085625
84461  5ltQ6  18.213839   2.191999  3.993869  107.813044
1305   LHZR0  11.170835  -1.945066  3.002872   80.859783
41906  LHZR0  -8.989672  -4.286607  2.009139   57.085625
2721   bfPNe  -9.494442  -5.463692  4.006042  110.992147
82178  bfPNe  -6.202799  -4.820045  2.995107   84.038886
47591  wt4Uk  -9.091098  -8.109279 -0.002314    3.179103
82873  wt4Uk  10.259972  -9.376355  4.994297  134.766305


In [12]:
data_2['id'].nunique()

99996

In [13]:
dublicate_2 = (
    data_2
    .pivot_table(index=('id'), values='f0', aggfunc='count')
    .reset_index()
    .query('f0 > 1')
)
dublicate_2 = dublicate_2['id'].tolist()
dublicate_2

['KUPhW', 'VF7Jo', 'Vcm5J', 'xCHr8']

In [14]:
print(data_2.query('id in @dublicate_2').sort_values(by = 'id'))

          id        f0        f1        f2     product
45404  KUPhW  0.231846 -1.698941  4.990775   11.716299
55967  KUPhW  1.211150  3.176408  5.543540  132.831802
11449  VF7Jo  2.122656 -0.858275  5.746001  181.716817
49564  VF7Jo -0.883115  0.560537  0.723601  136.233420
44378  Vcm5J -1.229484 -2.439204  1.222909  137.968290
95090  Vcm5J  2.587702  1.986875  2.482245   92.327572
28039  xCHr8  1.633027  0.368135 -2.378367    6.120525
43233  xCHr8 -0.847066  2.101796  5.597130  184.388641


В датасетах имеются дубликаты id. Изучив данные дубликаты, делаем вывод, что никакой связи между параметрами повторяющихся id нет. Следовательно оставляем датасет без изменения

# 2. Обучение и проверка модели

- Проверим разброс значений f0, f1, f2, для принятия решения о масштабировании

In [15]:
for data in data_0, data_1, data_2:
    print (data[['f0', 'f1', 'f2']].describe())

                  f0             f1             f2
count  100000.000000  100000.000000  100000.000000
mean        0.500419       0.250143       2.502647
std         0.871832       0.504433       3.248248
min        -1.408605      -0.848218     -12.088328
25%        -0.072580      -0.200881       0.287748
50%         0.502360       0.250252       2.515969
75%         1.073581       0.700646       4.715088
max         2.362331       1.343769      16.003790
                  f0             f1             f2
count  100000.000000  100000.000000  100000.000000
mean        1.141296      -4.796579       2.494541
std         8.965932       5.119872       1.703572
min       -31.609576     -26.358598      -0.018144
25%        -6.298551      -8.267985       1.000021
50%         1.153055      -4.813172       2.011479
75%         8.621015      -1.332816       3.999904
max        29.421755      18.734063       5.019721
                  f0             f1             f2
count  100000.000000  100000.00

Разброс значений небольшой, поэтому масштабирование не проводим

- Для исследований создадим функцию, разделяющую датасет на обучающую и валидационную выборки, обучающую модель и возвращающую предсказания и правильные ответы на валидационной выборке

In [16]:
def model(data):
    X_col = ['f0', 'f1', 'f2']
    y_col = ['product']
    X_train, X_valid, y_train, y_valid = train_test_split(
    data[X_col], data[y_col], test_size=0.25, random_state=42)
    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = pd.DataFrame(data=model.predict(X_valid), index=y_valid.index, columns=y_valid.columns)
    return y_pred, y_valid 

- Применим функцию к регионам data_0, data_1, data_2

In [17]:
y_pred_0, y_valid_0 = model(data_0)
y_pred_1, y_valid_1 = model(data_1)
y_pred_2, y_valid_2 = model(data_2)

- Выведем средний запас предсказанного сырья и RMSE

### data_0

In [18]:
def metrics(y_valid, y_pred):
    mse = mean_squared_error(y_valid, y_pred)
    print('Средний запас предсказанного сырья:', y_pred.mean())
    print('RMSE:', mse ** 0.5)
    return mse

In [19]:
mse_0 = metrics(y_valid_0, y_pred_0)

Средний запас предсказанного сырья: product    92.3988
dtype: float64
RMSE: 37.75660035026169


Рассчитаем RMSE вручную

In [20]:
MSE_0 = (y_pred_0 - y_valid_0) ** 2
RMSE_0 = (MSE_0.sum() / len(y_pred_0)) ** 0.5
RMSE_0 

product    37.7566
dtype: float64

In [21]:
print(mean_absolute_error(y_valid_0, y_pred_0))

31.047681538788183


### data_1

In [22]:
mse_1 = metrics(y_valid_1, y_pred_1)

Средний запас предсказанного сырья: product    68.712878
dtype: float64
RMSE: 0.8902801001028828


### data_2

In [23]:
mse_2 = metrics(y_valid_2, y_pred_2)

Средний запас предсказанного сырья: product    94.771024
dtype: float64
RMSE: 40.145872311342174


Для наглядности сведем результаты 3-х регионов в таблицу и добавим среднее значение запасов по всему региону

In [24]:
rezult = pd.DataFrame(
    {'регион': ['data_0', 'data_1', 'data_2'], 
     'сред_запас_пред': [y_pred_0['product'].mean(), y_pred_1['product'].mean(), y_pred_2['product'].mean()],
     'RMSE' : [mse_0 ** 0.5, mse_1 ** 0.5, mse_2 ** 0.5],
    'сред_запас_рег': [data_0['product'].mean(), data_1['product'].mean(), data_2['product'].mean()]})
rezult

,регион,сред_запас_пред,RMSE,сред_запас_рег
0,data_0,92.398800,37.756600,92.500
1,data_1,68.712878,0.890280,68.825
2,data_2,94.771024,40.145872,95.000


## Изучив результаты предсказаний, можно сделать следующие выводы:

1. Средние запасы региона_0 и региона_2 практически одинаковы, в регионе_1 запасов в среднем меньше
2. В ргионах 0 и 2 модель выдает большой RMSE, но сравнивая результаты с действительными значениями, результаты близки.

# 3. Подготовка к расчёту прибыли

Все ключевые значения для расчётов сохраним в отдельных переменных

- Бюджет на разработку скважин в регионе, тыс. руб.

In [25]:
budget = 10000000

- Доход с каждой единицы продукта, тыс. руб.

In [26]:
income = 450

- Количество скважин для расчета прибыли

In [27]:
wells = 200

- Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины

In [28]:
min_stock = budget / (income * wells)
min_stock

111.11111111111111

In [29]:
rezult[['регион', 'сред_запас_рег']]

,регион,сред_запас_рег
0,data_0,92.500
1,data_1,68.825
2,data_2,95.000


1. Для безубыточной разработки новой скважины минимальный запас сырья должен составлять 112 тыс.баррелей
2. Средние значения запасов в регионах меньше 112 тыс.баррелей, поэтому для анализа нужны лучшие скважины.

## Функция для расчёта прибыли по выбранным скважинам и предсказаниям модели

In [30]:
def revenue(target, predicted):
    pred_sorted = predicted.sort_values(ascending=False)
    selected = target[pred_sorted.index][:wells]
    selected_sum = selected.sum()
    return ((income * selected_sum) - budget)

## Комментарий наставника
<span style="color:green">Функция правильная.</span>

# 4. Расчёт прибыли и рисков 

- Рассчитаем прибыль и риски для региона 0

In [31]:
target_0 = y_valid_0['product']
predicted_0 = y_pred_0['product']
state = np.random.RandomState(42)
def calculations(target, predicted):
    values = []
    for i in range(1000):
        target_subsample = target.sample(n=500, replace=True, random_state=state)
        pred_subsample = predicted[target_subsample.index]
        values.append(revenue(target_subsample, pred_subsample))

    values = pd.Series(values)
    lower = values.quantile(0.025)
    upper = values.quantile(.975)

    mean = values.mean()

    loss = (values < 0).astype(int).sum() / len(values)

    print("Средняя прибыль, тыс.руб.:", mean)
    print("95%-ый доверительный интервал:", lower, upper)
    print("Риск убытков:", loss)
calculations(target_0, predicted_0)

Средняя прибыль, тыс.руб.: 427847.56046252465
95%-ый доверительный интервал: -97249.82956859532 954215.1927088142
Риск убытков: 0.055


- Рассчитаем прибыль и риски для региона 1

In [32]:
target_1 = y_valid_1['product']
predicted_1 = y_pred_1['product']
calculations(target_1, predicted_1)

Средняя прибыль, тыс.руб.: 511530.2181485244
95%-ый доверительный интервал: 91700.56413644394 921455.6683285085
Риск убытков: 0.006


- Рассчитаем прибыль и риски для региона 2

In [33]:
target_2 = y_valid_2['product']
predicted_2 = y_pred_2['product']
calculations(target_2, predicted_2)

Средняя прибыль, тыс.руб.: 408545.68121493567
95%-ый доверительный интервал: -120624.87294271737 960859.4407253832
Риск убытков: 0.075


# Выводы:

1. Лучше всего разработанная модель сработала в регионе_1 (data_1), несмотря на худшие показатели запасов сырья, благодаря максимальной точности предсказаний модели в данном регионе 
2. Средняя прибыль составила - 511 млн. руб.
3. 95%-ый доверительный интервал: от 92 млн. руб до  921 млн. руб 
4. Риск убытков составил 0,6 %, что удовлетворяет условию в 2,5% допустимой вероятности убытков
5. Для данной модели рекомендован регион_1 (data_1)